In [1]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from src import FactorialWeights

In [2]:
def gen_data(seed = 123, num_samples = 1000, model_type = 0, rho_cov = 0.15, error_type = 0, censoring = 0.15):
    df = pd.DataFrame()
    np.random.seed(seed)
    mu_cov = np.array([0.5, 0.5, 0.5, 0.5, 0.5])
    sigma_cov = np.full((5, 5), rho_cov)
    np.fill_diagonal(sigma_cov, 1)

    data = np.random.multivariate_normal(mu_cov, sigma_cov, num_samples)
    df[['X1', 'X2', 'X3', 'X4', 'X5']] = data

    beta_1 = np.array([-1/4, 3/4, 0, 2/4, -1])
    beta_2 = np.array([3/4, -1/4, -1, 0, 1/2])
    beta_3 = np.array([-1, 0, 3/4, 1/2, -1/4])
    betas = [beta_1, beta_2, beta_3]

    for i in range(len(betas)):
        beta = betas[i]
        prob = 1/(1+np.exp(-df[['X1', 'X2', 'X3', 'X4', 'X5']] @ beta))
        random_values = np.random.rand(len(df))
        df[f'Z{i+1}'] = (random_values < prob).astype(int)
        df[f'Z{i+1}'] = df[f'Z{i+1}'].replace(0, -1)                

    errors = 0
    if error_type == 0:
        errors = np.random.gumbel(0, 1, len(df))
    if error_type == 1:
        errors = np.random.normal(0, 1, len(df))
    if model_type == 0:
        df['Y'] = df[['X1', 'X2', 'X3', 'X4', 'X5']].sum(axis = 1) * 2
        df['Y'] += df[['Z1', 'Z2', 'Z3']].sum(axis = 1)
        #Mean is [0.4, 0.4, 0.4]
    if model_type == 1:
        df['Y'] = df[['X1', 'X2', 'X3', 'X4', 'X5']].sum(axis = 1)
        for i in ['X1', 'X2', 'X3', 'X4', 'X5']:
            for j in ['Z1', 'Z2', 'Z3']:
                df['Y'] += df[i] * df[j]
        #Mean is [1, 1, 1]
    if model_type == 2:
        df['Y'] = np.sin(df['X1'])
        df['Y'] += np.cos(df['X2'])
        df['Y'] += (np.minimum(1, df['X1']) + df['X2']) * df['Z1']
        for i in ['X1', 'X2', 'X3', 'X4', 'X5']:
            for j in ['Z2', 'Z3']:
                df['Y'] += df[i] * df[j] 

        #Mean is [0.3208, 1, 1]

    df['Y'] += errors
    df['T'] = np.exp(df['Y']/5)
    df = df.drop(columns = ['Y'])
    censored_max = np.random.uniform(low=0, high = 1/(2 * censoring), size=len(df))

    ranks = np.argsort(np.argsort(df['T']))
    quantiles = ranks / (len(df['T']) - 1)
    df['C'] = np.quantile(df['T'], np.minimum(censored_max, 1))
    df['delta'] = (quantiles <= censored_max).astype(int)
    df['tilde_T'] = np.log(np.minimum(df['T'], df['C']))
    return df

In [3]:
def impute_Y(ans):
    df = ans.copy()
    theta = np.array([0]*8)

    for i in range(100):
        df['cond_Y'] = df[['X1', 'X2', 'X3', 'X4', 'X5', 'Z1', 'Z2', 'Z3']] @ theta
        df['r_theta'] = df['tilde_T'] - df['cond_Y']
        df = df.sort_values('r_theta').reset_index(drop = True)
        
        n = len(df)
        frac = (n - df.index - 1)/(n - df.index)
        df['F'] = 1 - (np.power(frac, df['delta'])).cumprod()
        
        df['cond_e'] = (df['r_theta'] * df['F'].diff())/(1 - df['F'])
        df['cond_e'] = df['cond_e'].fillna(0)
        df['cond_e'] = df['cond_e'].replace([np.inf, -np.inf], 0)
    
        df = df.iloc[::-1].reset_index(drop=True)
        length = (df['cond_e'].max() - df['cond_e'])
        df['cond_e'] = df['cond_e'].cumsum().shift(1)/(length*3)
        df['cond_e'] = df['cond_e'].fillna(df['cond_e'].mean())
        D = df[['X1', 'X2', 'X3', 'X4', 'X5', 'Z1', 'Z2', 'Z3']]
        D_mean = D.mean(axis = 0)
        D_vec = D - D_mean
        inverse = 1/(D_vec**2).sum(axis=1).sum()
        
        df['Y_impute'] = df['tilde_T']
        
        df.loc[df['delta'] == 0, 'Y_impute'] = df.loc[df['delta'] == 0, 'cond_Y'] + df.loc[df['delta'] == 0, 'cond_e']
        df['Y_impute'] = np.maximum(df['Y_impute'], df['tilde_T'])
        
        Y_vec = df['Y_impute'] - df['Y_impute'].max()
        theta = inverse * (D_vec.values * np.array(Y_vec)[:, np.newaxis]).sum(axis = 0)
    
    df = df.drop(columns = ['T', 'C', 'cond_Y', 'r_theta', 'F', 'cond_e'])
    return df

In [4]:
def generate_single_test(random_seed, model_type, error_type, censoring):
    df = gen_data(seed = random_seed, model_type = model_type, error_type = error_type, censoring = censoring)
    df = impute_Y(df)
    df['X0'] = 1

    fw = FactorialWeights()
    treat_cols = ['Z1', 'Z2', 'Z3']
    cov_cols = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5']

    naive_estimators = []
    for effect in ['Z1', 'Z2', 'Z3']:
        ATE = df.loc[df[effect] == 1, 'tilde_T'].mean() - df.loc[df[effect] == -1, 'tilde_T'].mean()
        ATE = float(ATE)
        naive_estimators.append(ATE)

    additive_estimators = [] 
    for effect in ['Z1', 'Z2', 'Z3']:
        w = fw.additive_treat_model(df, treat_cols, cov_cols, [effect], weighting = 'variance')
        ATE =  (df[effect] * w * df['Y_impute']).sum()/len(df)
        ATE = float(ATE)
        additive_estimators.append(ATE)

    hetero_estimators = []
    for effect in ['Z1', 'Z2', 'Z3']:
        w = fw.heterogeneous_treat_model(df, treat_cols, cov_cols, [effect], weighting = 'variance')
        ATE =  (df[effect] * w * df['Y_impute']).sum()/len(df)
        ATE = float(ATE)
        hetero_estimators.append(ATE)

    return naive_estimators, additive_estimators, hetero_estimators
    

In [5]:
def generate_test_suite(model_type, error_type, censoring, num_sim = 1000):
    
    fairs = {}
    if model_type == 0:
        fairs = {'Z1':0.4, 'Z2': 0.4, 'Z3': 0.4}
    if model_type == 1:
        fairs = {'Z1':1, 'Z2':1, 'Z3':1}
    if model_type == 2:
        fairs = {'Z1':0.3208, 'Z2':1, 'Z3':1}

    results = {}
    estimators = ['naive', 'additive', 'hetero']
    estimands = ['Z1', 'Z2', 'Z3']

    for estimator in estimators:
        for estimand in estimands:
            results[(estimator, estimand)] = []
    
    for sim in range(num_sim):
        random_seed = np.random.randint(0, 10_000_000)
        
        e1, e2, e3 = generate_single_test(random_seed, model_type, error_type, censoring)

        results[('naive', 'Z1')].append(e1[0])
        results[('naive', 'Z2')].append(e1[1])
        results[('naive', 'Z3')].append(e1[2])
        results[('additive', 'Z1')].append(e2[0])
        results[('additive', 'Z2')].append(e2[1])
        results[('additive', 'Z3')].append(e2[2])
        results[('hetero', 'Z1')].append(e3[0])
        results[('hetero', 'Z2')].append(e3[1])
        results[('hetero', 'Z3')].append(e3[2])

        if sim % 100 == 0:
            print(sim)

    test_results = {}
    for estimator in estimators:
        for estimand in estimands:
            arr = np.array(results[(estimator, estimand)])
            bias = float(arr.mean() - fairs[estimand])
            rmse = float(np.sqrt(((arr- fairs[estimand])**2).mean()))
            test_results[(estimator, estimand)] = (bias, rmse)

    test_results['setting'] = {'model_type':model_type, 'error_type':error_type, 'censoring':censoring}
    return test_results

In [6]:
test_res = generate_test_suite(0, 0, 0.15, 10)

0


In [7]:
model_types = [0, 1, 2]
error_types = [0, 1] 
censoring = [0.15, 0.25]

In [8]:
test_res

{('naive', 'Z1'): (-0.09440690781186323, 0.10931146570208816),
 ('naive', 'Z2'): (-0.13945935011881033, 0.17293482600319132),
 ('naive', 'Z3'): (-0.12390828506809848, 0.14727345768480238),
 ('additive', 'Z1'): (-0.031809780846179025, 0.039574686688239595),
 ('additive', 'Z2'): (-0.044807314710182844, 0.051671454993808555),
 ('additive', 'Z3'): (-0.04179384798989283, 0.04420254384420213),
 ('hetero', 'Z1'): (-0.042257265350834905, 0.047963494945486095),
 ('hetero', 'Z2'): (-0.048617263842227476, 0.05486813960582897),
 ('hetero', 'Z3'): (-0.04777702947932777, 0.049382866054329656),
 'setting': {'model_type': 0, 'error_type': 0, 'censoring': 0.15}}

In [11]:
df = gen_data()
df = impute_Y(df)

df_small = df.loc[df['delta'] == 0].copy()

df_small['Y_impute'].mean(), df['Y_impute'].mean()

(np.float64(1.8866744219136813), np.float64(1.101382064357695))